# Title:LapTrack

## Problem Statement:

### Analysis of laptops across various platforms to understand the factors influencing the price and specifications influencing customers interest in purchasing a laptop.
### Through this analysis we can find out the elements that are important in fixing the prices, and the elements that makes the major contribution in the price. Also finding the main features of the laptop that are prior requirements of the customers , and which are making a clear impression on the product to the customer.
### This analysis will help both seller and customer to obtain the best beniefits as for seller the information about the requirements and the first things that customer observe while before the buying the laptop will help making better prices and better products according to the customers requirements.For customers it would be helpful in chossing the right product acording to their price range and the best features that can be obtained.

### For this we decieded to obtain data from online ecommerce weebsites Amazon,Flipkart,BestBuy

## Questions

## 1.How Rating is influencing customers while selecting a laptop?
## Through this we can find the influence of the rating factors in both seller and customer way
## One of the major factor that influence customers is rating because rating is given by the customers who already have the past experience with the product.Through this customers can analyse the product .Here we can observe whether rating is an impoertant factor in buying the product.

## 2.How display size is influcing customers?
## What size customers prefer most 
## This is also an imoirtant factor because according to the purpose of their usage customers buy laptops with respective size.Through this we can know what size customers prefer more while buying them online

## Data Retrieval  : Best Buy

## Scraping Data

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to get the HTML content of a page
def get_page_content(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.content, 'html.parser')

# Function to scrape laptop data from one page
def scrape_laptop_data_from_page(soup, data):
    base_url = 'https://www.bestbuy.com'

    # Find all laptops on the page
    laptops = soup.find_all('li', class_='sku-item')

    for laptop in laptops:
        try:
            # Extract laptop name
            name_tag = laptop.find('h4', class_='sku-title')
            name = name_tag.text.strip() if name_tag else 'No name available'

            # Extract SKU value
            skuvalue_tag = laptop.find('span', class_='sku-value')
            skuvalue = skuvalue_tag.text.strip() if skuvalue_tag else 'No SKU value available'

            # Extract rating (visually hidden text)
            visually_hidden_tag = laptop.find('p', class_='visually-hidden')
            rating = visually_hidden_tag.text.strip() if visually_hidden_tag else 'No rating available'

            # Extract laptop price
            price_tag = laptop.find('div', class_='priceView-customer-price')
            price = price_tag.span.text.strip() if price_tag else 'No price available'



            # Append data to the list
            data.append([name, skuvalue, rating, price])
            #print(f'Scraped: {name}')

        except Exception as e:
           print(f'Error scraping laptop: {e}')

# Function to find the link to the next page
def get_next_page(soup):
    next_page_tag = soup.find('a', class_='sku-list-page-next')
    if next_page_tag and 'href' in next_page_tag.attrs:
        return next_page_tag['href']
    return None

# Main function to scrape all pages
def scrape_all_pages():
    base_url = 'https://www.bestbuy.com'
    search_url = f'{base_url}/site/searchpage.jsp?st=laptops'

    # List to hold all laptop data
    data = []

    current_page_url = search_url

    while current_page_url:
        #print(f'Scraping page: {current_page_url}')
        soup = get_page_content(current_page_url)
        scrape_laptop_data_from_page(soup, data)

        # Check if there's a next page
        next_page = get_next_page(soup)
        if next_page:
            current_page_url = f'{base_url}{next_page}'
        else:
            current_page_url = None

    # Create a DataFrame from the scraped data
    df = pd.DataFrame(data, columns=['total_info', 'Model No', 'Rating', 'Price'])

    # Save DataFrame to a CSV file
    df.to_csv('laptops_data.csv', index=False)
    print('Data has been saved to laptops_data1.csv')

    # Set display options to show all rows and columns
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.expand_frame_repr', False)  # Do not wrap columns
    pd.set_option('display.max_colwidth', None)  # Show full column width

    # Print the DataFrame to the console
    return df

if __name__ == '__main__':
    df=scrape_all_pages()
    #print(df)
print(df.info())

Data has been saved to laptops_data1.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   total_info  1284 non-null   object
 1   Model No    1284 non-null   object
 2   Rating      1284 non-null   object
 3   Price       1284 non-null   object
dtypes: object(4)
memory usage: 40.3+ KB
None


In [35]:
#Extracting Columns
import pandas as pd
#Brand
df['Brand']='-'
df['Brand'] = df['total_info'].str.split(' ').str[0]
#Colur
df['Colour']='-'
df['Colour'] = df['total_info'].str.split(' ').str[-1]
#Saving original
df1=df
#Remove Special Characters
df1['total_info'] = df1['total_info'].str.replace('”', '"', regex=False)
#Ram
ram_pattern = r'(\s*\d+\s*G[Bb]\s*(Memory|RAM|Ram)\s*|(\d{1,2}GB))'
df1['ram'] = df1['total_info'].str.extract(ram_pattern, expand=False)[0]
df1['ram'] = df1['ram'].str.extract(r'(\d+)').astype(float)
df1['ram'] = df1['ram'].apply(lambda x: f"{int(x)} GB" if x < 33 else pd.NA)
#Storage
storage_pattern = r'(\d+\s*(TB|GB|G)\s*(SSD|HDD|Solid State Drive|Flash Storage|Hard Drive|eMMC|UFS|SDD|PCIe|NVMe|Storage)|\d+\s*TB\s*-\s*SSD|\d{3,4}SSD|\d{3,4}GB\s*-\s*SSD)'
df1['storage'] = df1['total_info'].str.extract(storage_pattern, expand=False)[0]
#Processor
processor_pattern = r'(Intel\s+\w+\s*\w*|\bM[12]\s+(?:Pro|Max|chip)\s*Built\s*for\s*Apple\s*-\w+|M[12]\s+(?:Pro|Max|chip)\b|M3\s+chip\s*Built\s*for\s*Apple\s*Intelligence|M3\s+\w+\s+chip\s*Built\s*for\s*Apple|M3\s+chip|AMD\s+Ryzen\s+\d+(?:-\d+|\s+\d+)\w*|AMD\s+Ryzen\s+\d+\b|AMD\s+Ryzen\s+AI\s+\d+-\d+\s+\w*|Apple\s+M1\s+\w+\s+chip\b|Core\s+\w+\s*-\d+|\bRyzen\s+\d+\s+\d+\w*|MediaTek\s+\w+\s+\d+|Snapdragon\s+\w+\s*\w*|Pentium\s+\w+\s+\d+)'
extracted_processors = df1['total_info'].str.extract(processor_pattern)
df1['processor'] = extracted_processors[0].fillna(value=pd.NA)
#Display size
display_pattern=r'(\d{2}"|\d{2}.\d"|\d{2}-inch|\d{2}-Inch|\d{2}.\d-inch|\d{2}.\d-Inch)'
extracted_displays = df1['total_info'].str.extract(display_pattern)
df1['display'] = extracted_displays[0].fillna(value=pd.NA)
#Laptop Name
model_pattern = r'(Geek Squad Certified Refurbished MacBook Air|Envy|XPS|OmniBook|ProBook|Flex|LOQ|Katana|Blade|Aero|Vector|Summit|Raider|Pulse|Cyborg|Elitebook|Precision|Galaxy book|Geek Squad Certified Refurbished Macbook®|Geek Squad Certified Refurbished MacBook Pro|MacBook Pro|GSRF MacBook Pro|MacBook Air|Refurbished MacBook®|Chromebook|ProArt P16|ProArt Studiobook|ROG Strix G16|ROG Strix SCAR|ROG Zephyrus G14|TUF A15|TUF Gaming A17|Vivobook|Vivobook Pro 15|Zenbook|Zenbook Pro Duo|Zenbook DUO Dual|Nitro 16|Predator Helios 18|Predator Triton|Swift X|TravelMate|m18 R2|m16|m18 165|Latitude 7000|Refurbished|Inspiron|Precision 3540|XPS 13|Latitude 3000|Mobile Processor|Alienware x14|Gaming Laptop IPS|EliteBook 840 G8|Elite x360 830 G11|Envy 2-in-1|Pavilion|Victus|ZBook|OMEN|gram|gram SuperSlim|ThinkPad X1|Yoga|Yoga Book|Yoga Pro|Legion|Ideapad|ThinkPad T14s|GSRF Surface|Surface Laptop - Copilot\+ PC|Surface Laptop Studio 2|Bravo 15|Commercial 14 H A13MG|Creator 17|Crosshair|Raider GE78 HX|Stealth|Summit E16 AI Studio|Vector 16 HX A14V|Blade 16|Galaxy Book2 Pro|Galaxy Book3 360|Galaxy Book4 Ultra|Geek Squad Certi Refurbis Galaxy Book Flex2 Alpha|Galaxy Book4 Edge|NEOZ3 Laptop|Galaxy Book4 Pro 360|Blade 18|ROG Zephyrus|Summit E16 AI Evo|Raider GE68 HX|Prestige|Modern 15|ThinkPad|ThinkPad T14|ThinkBook 14 G7|Spectre|AORUS|Latitude 5000|Latitude|Predator Helios|Aspire 5|ProArt PX13 13|Blade 15|ProBook 445|EliteBook 640|Raider 18|CreatorPro|Creator 16|ThinkBook|Swift 14|ROG Flow)'
df1['model'] = df1['total_info'].str.extract(model_pattern)
#Graphics
graphics_pattern = r'(NVIDIA GeForce RTX \d+|Ryzen \d+U|Intel Iris Xe Graphics|NVIDIA Quadro P1000|Intel UHD Graphics 620|NVIDIA Quadro T1000|AMD Radeon Graphics|NVIDIA GeForce RTX4070|NVIDIA Quadro P3200)'
df1['graphics'] = df1['total_info'].str.extract(graphics_pattern)
#Storage Type
storagetype_pattern = r'(FlashStorage|HardDrive|SSD|HDD|SDD|eMMC|Flash Storage|Hard Drive|NVMe|PCIe|UFS|Solid State Drive)'
df1['storage_type'] = df1['storage'].str.extract(storagetype_pattern)
#No of reviews
no_reviews_pattern = r'(\d{1,4})\s*reviews?|reviewfalse'
df1['no_reviews'] = df1['Rating'].str.extract(no_reviews_pattern)
#Rating out of 5
Rating_5_pattern = r'Rating\s*(\d(?:\.\d)?)'
df1['Rating_5'] = df1['Rating'].str.extract(Rating_5_pattern)
#Processor Company
processor_company_pattern = r'(Intel|AMD\s*Ryzen|Ryzen|Snapdragon|Core|MediaTek|M1|M2|M3)'
df1['processor_company'] = df1['processor'].str.extract(processor_company_pattern)
#Operating System
df1['os']='Windows11'
#Dropping group columns
df1 = df1.drop('total_info', axis=1)
df1 = df1.drop('Rating', axis=1)
#Saving original
df_BB=df1

In [37]:
#Filling null values
df_BB['graphics'] = df_BB['graphics'].fillna('No Graphics')

df_BB['no_reviews'] = df_BB['no_reviews'].fillna('0')

df_BB['Rating_5'] = df_BB['Rating_5'].fillna('0 Reviews')

df_BB['storage_type'] = df_BB['storage_type'].fillna('SSD')

df_BB['processor'] = df_BB['processor'].fillna('No Info')

df_BB['processor_company'] = df_BB['processor_company'].fillna('No Info')

df_BB['model'] = df_BB['model'].fillna('No Model')

most_frequent_display = df_BB['display'].mode()[0]
df_BB['display'] = df_BB['display'].fillna(most_frequent_display)

most_frequent_ram = df_BB['ram'].mode()[0]
df_BB['ram'] = df_BB['ram'].fillna(most_frequent_ram)

most_frequent_storage = df_BB['storage'].mode()[0]
df_BB['storage'] = df_BB['storage'].fillna(most_frequent_storage)

df_BB['ram'] = df_BB['ram'].str.replace('GB', '')
df_BB['ram'] = df_BB['ram'].str.replace(' ', '')





In [39]:
#More cleaning

df_BB.loc[df_BB['Brand'] == 'Apple', 'os'] = 'Mac OS'

df_BB['processor_company'] = df_BB['processor_company'].replace({'M1': 'Apple', 'M2': 'Apple', 'M3': 'Apple','Core':'Intel','Ryzen': 'AMD Ryzen'})

df_BB['processor'] = df_BB['processor'].str.replace('Intel', '')
df_BB['processor'] = df_BB['processor'].str.replace('AMD Ryzen', '')
df_BB['processor'] = df_BB['processor'].str.replace('Snapdragon', '')
df_BB['processor'] = df_BB['processor'].str.replace('MediaTek', '')
df_BB['processor'] = df_BB['processor'].str.replace('Ryzen', '')

df_BB['Brand'] = df_BB['Brand'].str.replace('New!', '')

df_BB['display'] = df_BB['display'].str.replace('"', '')

df_BB['storage'] = df_BB['storage'].str.replace('GB', '')
df_BB['storage'] = df_BB['storage'].str.replace('SSD', '')
df_BB['storage'] = df_BB['storage'].str.replace('Solid State Drive', '')
df_BB['storage'] = df_BB['storage'].str.replace('SDD', '')
df_BB['storage'] = df_BB['storage'].str.replace('HDD', '')
df_BB['storage'] = df_BB['storage'].str.replace('TB PCIe', 'TB')
df_BB['storage'] = df_BB['storage'].str.replace('-', '')
df_BB['storage'] = df_BB['storage'].str.replace(' ', '')
df_BB['storage'] = df_BB['storage'].str.replace('HardDrive', '')
df_BB['storage'] = df_BB['storage'].str.replace('NVMe', '')
df_BB['storage'] = df_BB['storage'].str.replace('FlashStorage', '')
df_BB['storage'] = df_BB['storage'].str.replace('TB NVMe', 'TB')
df_BB['storage'] = df_BB['storage'].str.replace('Storage', '')
def convert_tb_to_gb(storage_value):
    if 'TB' in storage_value:
        
        tb_value = float(storage_value.replace('TB', '').strip())
        return str(int(tb_value * 1024))
    return storage_value
df_BB['storage'] = df_BB['storage'].apply(convert_tb_to_gb)

df_BB['storage_type'] = df_BB['storage_type'].str.replace('Solid State Drive', 'SSD')

df_BB['ram'] = df_BB['ram'].str.replace('GB', '')
df_BB['ram'] = df_BB['ram'].str.replace(' ', '')

#Merging Model columns
df_BB['processor_model'] = df_BB['processor'] + ' - ' + df_BB['Model No']
df_BB = df_BB.drop(['processor', 'Model No'], axis=1)

df_BB['Price'] = df_BB['Price'].str.replace('$', '')
df_BB['Price'] = df_BB['Price'].str.replace(',', '')

df_BB['display'] = df_BB['display'].str.replace('-Inch', '')

In [87]:
df_BB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Laptop_Price            1284 non-null   object
 1   Laptop_Brand            1284 non-null   object
 2   Laptop_Colour           1284 non-null   object
 3   Laptop_Memory_GB        1284 non-null   object
 4   Laptop_Storage_GB       1284 non-null   object
 5   Laptop_Display_Size_in  1284 non-null   object
 6   Laptop_name             1284 non-null   object
 7   Laptop_Graphics         1284 non-null   object
 8   No_Of_Reviews           1284 non-null   object
 9   Rating_5                1284 non-null   object
 10  Processor_Company_Name  1284 non-null   object
 11  Storage_Type            1284 non-null   object
 12  Operating_System        1284 non-null   object
 13  Processor_Model         1284 non-null   object
dtypes: object(14)
memory usage: 140.6+ KB


In [86]:
#Renameing columns for better understading
df_BB = df_BB.rename(columns={'model': 'Laptop_name'})
df_BB = df_BB.rename(columns={'Brand': 'Laptop_Brand'})
df_BB = df_BB.rename(columns={'Colour': 'Laptop_Colour'})
df_BB = df_BB.rename(columns={'ram': 'Laptop_Memory_GB'})
df_BB = df_BB.rename(columns={'storage': 'Laptop_Storage_GB'})
df_BB = df_BB.rename(columns={'display': 'Laptop_Display_Size_in'})
df_BB = df_BB.rename(columns={'graphics': 'Laptop_Graphics'})
df_BB = df_BB.rename(columns={'no_reviews': 'No_of_Reviews'})
df_BB = df_BB.rename(columns={'processor_company': 'Processor_Company_Name'})
df_BB = df_BB.rename(columns={'storage_type': 'Storage_Type'})
df_BB = df_BB.rename(columns={'os': 'Operating_System'})
df_BB = df_BB.rename(columns={'processor_model': 'Processor_Model'})

In [88]:
df_BB.to_csv('BestBuy.csv', index=False)